In [ ]:
# model definition, seq2seq stateful lstm for real-time audio processing

import torch
import torch.nn as nn

class AudioLSTM(nn.Module):
  def __init__(self, input_size, output_size, hidden_size, num_layers):
    super(AudioLSTM, self).__init__()
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size, output_size)

    # needed for stateful LSTM
    self.hidden = None

  def forward(self, x):
    if self.hidden is None:
      self.hidden = (
        torch.zeros(num_layers, x.size(0), hidden_size).to(x.device),
        torch.zeros(num_layers, x.size(0), hidden_size).to(x.device)
      )

    out, self.hidden = self.lstm(x, self.hidden)

    out = self.fc(out)
    return out
  
  def reset_hidden(self):
    self.hidden = None

In [ ]:
# custom dataloader, feed 1 sample at a time

import torchaudio
